In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('CelularesSubtraidos_2025.xlsx', sheet_name=0)

In [3]:
df.columns

Index(['ID_DELEGACIA', 'NOME_DEPARTAMENTO', 'NOME_SECCIONAL', 'NOME_DELEGACIA',
       'NOME_MUNICIPIO', 'ANO_BO', 'NUM_BO', 'VERSAO',
       'NOME_DEPARTAMENTO_CIRC', 'NOME_SECCIONAL_CIRC', 'NOME_DELEGACIA_CIRC',
       'NOME_MUNICIPIO_CIRC', 'DATA_OCORRENCIA_BO', 'HORA_OCORRENCIA',
       'DESCRICAO_APRESENTACAO', 'DATAHORA_REGISTRO_BO', 'DATA_COMUNICACAO_BO',
       'DATAHORA_IMPRESSAO_BO', 'DESCR_PERIODO', 'AUTORIA_BO',
       'FLAG_INTOLERANCIA', 'TIPO_INTOLERANCIA', 'FLAG_FLAGRANTE',
       'FLAG_STATUS', 'DESC_LEI', 'FLAG_ATO_INFRACIONAL', 'RUBRICA',
       'DESCR_CONDUTA', 'DESDOBRAMENTO', 'CIRCUNSTANCIA', 'DESCR_TIPOLOCAL',
       'DESCR_SUBTIPOLOCAL', 'CIDADE', 'BAIRRO', 'CEP', 'LOGRADOURO_VERSAO',
       'LOGRADOURO', 'NUMERO_LOGRADOURO', 'LATITUDE', 'LONGITUDE',
       'CONT_OBJETO', 'DESCR_MODO_OBJETO', 'DESCR_TIPO_OBJETO',
       'DESCR_SUBTIPO_OBJETO', 'DESCR_UNIDADE', 'QUANTIDADE_OBJETO',
       'MARCA_OBJETO', 'FLAG_BLOQUEIO', 'FLAG_DESBLOQUEIO', 'MES', 'ANO'],
      d

In [4]:
# Creating a copy
df_original = df
df = df_original

### Cleaning:
- Dropped rows with missing coordinates or report number
- Filtered for cellphone-related robberies only
- Removed duplicates by report number
- Kept only incidents from 2025
- Filtered to São Paulo city only

In [5]:
len(df)

132640

In [6]:
cellphone_robbery = [
    "Roubo (art. 157)",                    # Robbery with violence or threat
    "Furto (art. 155)",                    # Theft without violence
    "Furto de coisa comum (art. 156)",     # Theft of jointly owned property
    "Receptação (art. 180)",               # Possession or sale of stolen property
    "Estelionato (art. 171)",              # Fraud (e.g., fake sale or scam involving a phone)
    "Extorsão (art. 158)",                 # Extortion (threats for gain)
    "Extorsão mediante seqüestro (art. 159)"  # Kidnapping with ransom/extortion
]

df = df[df['RUBRICA'].isin(cellphone_robbery)]
len(df)

109097

In [7]:
df = df.dropna(subset=['LATITUDE', 'LONGITUDE', 'NUM_BO'])
len(df)

95229

In [8]:
df = df.drop_duplicates(subset=['NUM_BO'])
len(df)

75563

In [9]:
df = df.query('ANO_BO == 2025')
len(df)

75545

In [10]:
df = df.query('CIDADE == "S.PAULO"')
len(df)

46030

### Reducing places categories

In [11]:
df['DESCR_CONDUTA'].unique()

array(['Outros', 'Interior de Veículo', 'Transeunte', 'Residência', nan,
       'Interior Transporte Coletivo', 'Condomínio Residêncial',
       'Interior Estabelecimento', 'Veículo', 'Estabelecimento Comercial',
       'Aplicativo de Mobilidade Urbana', 'Carga', 'Coletivo',
       'Estabelecimento-Outros', 'Pessoa', 'Condomínio Comercial',
       'Saidinha de Banco', 'Estabelecimento Ensino', 'Caixa Eletrônico',
       'Joalheria'], dtype=object)

In [12]:
descr_conduta_map_reduce = {
    "Outros": "Other",
    "NULL": "Other",
    
    "Residência": "Residential Area",
    "Condomínio Residêncial": "Residential Area",

    "Interior de Veículo": "Transport",
    "Veículo": "Transport",
    "Interior Transporte Coletivo": "Transport",
    "Coletivo": "Transport",
    "Aplicativo de Mobilidade Urbana": "Transport",

    "Transeunte": "Public Space",
    "Pessoa": "Public Space",
    "Carga": "Public Space",
    "Fios e Cabos": "Public Space",

    "Interior Estabelecimento": "Establishment",
    "Estabelecimento Comercial": "Establishment",
    "Estabelecimento-Outros": "Establishment",
    "Condomínio Comercial": "Establishment",
    "Estabelecimento Ensino": "Establishment",
    "Joalheria": "Establishment",

    "Estabelecimento Bancário (Roubo/Furto a Banco)": "Banking",
    "Caixa Eletrônico": "Banking",
    "Saidinha de Banco": "Banking"
}


In [13]:
df['Location_Type'] = df['DESCR_CONDUTA'].map(descr_conduta_map_reduce).fillna("Other")

### Selecting columns

In [14]:
df = df[[
    'MES',
    'LATITUDE',
    'LONGITUDE',
    'BAIRRO',
    'Location_Type',
    'DESCR_TIPOLOCAL',
    'DESCR_SUBTIPOLOCAL'
]]

### Transforming in GeoJson

In [15]:
import geopandas as gpd
from shapely.geometry import Point

In [16]:
df['geometry'] = df.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)

In [17]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

In [18]:
gdf.to_file("robberies_sp_25.geojson", driver="GeoJSON")